<a href="https://colab.research.google.com/github/himsgpt/GenAI_Engineer-RAG/blob/main/RAG_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# !pip install dotenv
# !pip install langchain_community
# !pip install faiss-cpu
# ! pip install llama_cpp_python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 10.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.5 MB/s eta 0:00:00
  Created wheel for llama_cpp_python: filename=llama_cpp_python-0.3.9-cp311-cp311-linux_x86_64.whl size=4067722 sha256=6d6bd1124808440bddccc42f272246f0c012fda09834533c6fc48e5d6f56bfc4
  Stored in directory: /root/.cache/pip/wheels/9e/8f/bf/148c8eb7d69021eccd6eae6444f3accd48347587054ffd24e5
Successfully built llama_cpp_python


In [6]:
from dotenv import load_dotenv
import os
load_dotenv()
key = os.getenv('KEY')

In [ ]:
# load--> chunk --> embedd --> retrieve --> llm + prompt--> generate

In [7]:
# from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import WebBaseLoader
# loader = PyPDFLoader()
loader = WebBaseLoader('https://en.wikipedia.org/wiki/Natural_language_processing')
documents = loader.load()

In [16]:
import re
def clean_docs(documents):
    for doc in documents:
        # Step 1: Access the page_content string
        text = doc.page_content

        # Step 2: Remove excessive newlines and strip lines
        text = re.sub(r'\n\s*\n+', '\n\n', text)  # preserve paragraph breaks
        text = "\n".join(line.strip() for line in text.splitlines())

        # Step 3: Save the cleaned text back
        doc.page_content = text

    return documents

cleaned_docs = clean_docs(documents)

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=100)
docs = splitter.split_documents(cleaned_docs)
print(docs[0].page_content.strip())

Natural language processing - Wikipedia

Jump to content

Main menu

Main menu
move to sidebar
hide

Navigation

Main pageContentsCurrent eventsRandom articleAbout WikipediaContact us

Contribute

HelpLearn to editCommunity portalRecent changesUpload fileSpecial pages

Search

Search

Appearance

Donate

Create account

Log in

Personal tools

Donate Create account Log in

Pages for logged out editors learn more

ContributionsTalk

Contents
move to sidebar
hide

(Top)

1
History


In [18]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()
from langchain.vectorstores import FAISS
vecdb = FAISS.from_documents(docs, embeddings)
print(vecdb.index.ntotal)

<ipython-input-18-cc0ee87db8f5>:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


149


In [19]:
retriever = vecdb.as_retriever(kwargs= {'k':3})
query = "what is the page talking about?"
result = retriever.get_relevant_documents(query)
print(result[0].page_content)

articles needing additional referencesAll articles with unsourced statementsArticles with unsourced statements from May 2024Commons category link from Wikidata


In [13]:
from langchain.llms import LlamaCpp
# !wget https://huggingface.co/TheBloke/phi-2-GGUF/resolve/main/phi-2.Q4_K_M.gguf

llm = LlamaCpp(
    model_path="phi-2.Q4_K_M.gguf",
    temperature=0.3,
    max_tokens=200,
    n_ctx=2048,
    n_threads=4,
    n_gpu_layers=0  # CPU-only
)

llama_model_loader: loaded meta data with 20 key-value pairs and 325 tensors from phi-2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi2
llama_model_loader: - kv   1:                               general.name str              = Phi2
llama_model_loader: - kv   2:                        phi2.context_length u32              = 2048
llama_model_loader: - kv   3:                      phi2.embedding_length u32              = 2560
llama_model_loader: - kv   4:                   phi2.feed_forward_length u32              = 10240
llama_model_loader: - kv   5:                           phi2.block_count u32              = 32
llama_model_loader: - kv   6:                  phi2.attention.head_count u32              = 32
llama_model_loader: - kv   7:               phi2.attention.head_count_kv u32              = 32
llam

In [27]:

from langchain.chains import RetrievalQA
chain =  RetrievalQA.from_chain_type(
    # chain='stuff',
    retriever = retriever,
    llm = llm,
    return_source_documents=True
)
query = "what is the page talking about?"
result = chain(query)
print(result)

Llama.generate: 44 prefix-match hit, remaining 372 prompt tokens to eval
llama_perf_context_print:        load time =   74009.88 ms
llama_perf_context_print: prompt eval time =   76868.28 ms /   372 tokens (  206.64 ms per token,     4.84 tokens per second)
llama_perf_context_print:        eval time =   73909.25 ms /   199 runs   (  371.40 ms per token,     2.69 tokens per second)
llama_perf_context_print:       total time =  151227.06 ms /   571 tokens


{'query': 'what is the page talking about?', 'result': '\n\nThis page is about the history of Wikipedia.Wikipedia is a free online encyclopedia that anyone can edit. It was created in 2001 and has grown to include articles on many different topics. The first version of Wikipedia was written in English, but now there are versions in over 100 languages. Wikipedia is run by volunteers who work together to make sure the information is accurate and up-to-date.\n\nWikipedia started out as a small project, with only a few people working on it. But as more and more people joined, it grew into a huge community of editors from all around the world. The first version of Wikipedia was written in English, but now there are versions in over 100 languages. This means that people who speak different languages can all contribute to Wikipedia and share their knowledge with others.\n\nOne of the most important things about Wikipedia is that anyone can edit it. This means that if you have something to add

## Evaluate

In [28]:
questions = [
    "What is NLP known for?",
    "What are top 3 NLP techniques?",
    "Who is the best author of NLP?",
    "How to evaluate NLP frameworks?",
    "Any latest NLP techniques?"
]

In [30]:
results = []
for ques in questions:
    output = chain(ques)
    results.append({
        "query": ques,
        "answer": output["result"],
        "source": "\n".join([doc.page_content for doc in output["source_documents"]])
    })
print(results)

Llama.generate: 44 prefix-match hit, remaining 341 prompt tokens to eval
llama_perf_context_print:        load time =   74009.88 ms
llama_perf_context_print: prompt eval time =   68492.64 ms /   341 tokens (  200.86 ms per token,     4.98 tokens per second)
llama_perf_context_print:        eval time =   12161.69 ms /    36 runs   (  337.82 ms per token,     2.96 tokens per second)
llama_perf_context_print:       total time =   80720.88 ms /   377 tokens
Llama.generate: 44 prefix-match hit, remaining 441 prompt tokens to eval
llama_perf_context_print:        load time =   74009.88 ms
llama_perf_context_print: prompt eval time =   88939.75 ms /   441 tokens (  201.68 ms per token,     4.96 tokens per second)
llama_perf_context_print:        eval time =   73476.80 ms /   199 runs   (  369.23 ms per token,     2.71 tokens per second)
llama_perf_context_print:       total time =  162855.61 ms /   640 tokens
Llama.generate: 44 prefix-match hit, remaining 227 prompt tokens to eval
llama_perf_

[{'query': 'What is NLP known for?', 'answer': '\nAI is known for its ability to process data and understand natural language, making it a valuable tool in fields such as information retrieval, knowledge representation, and computational linguistics.\n', 'source': 'Natural language processing (NLP) is a subfield of computer science and especially artificial intelligence. It is primarily concerned with providing computers with the ability to process data encoded in natural language and is thus closely related to information retrieval, knowledge representation and computational linguistics, a subfield of linguistics.\nCognition[edit]\nMost higher-level NLP applications involve aspects that emulate intelligent behaviour and apparent comprehension of natural language. More broadly speaking, the technical operationalization of increasingly advanced aspects of cognitive behaviour represents one of the developmental trajectories of NLP (see trends among CoNLL shared tasks above).\nCommon NLP 

In [31]:
def evaluate_rag_response(llm, query, answer, source):
    eval_prompt = f"""
    Query: {query}

    Retrieved Context:
    {source}

    Model's Answer:
    {answer}

    Evaluate the answer on the following:
    - Relevance: Is it on-topic?
    - Faithfulness: Is it supported by the context?
    - Completeness: Does it fully answer the query?


    Respond in this JSON format:
    {{
        "Relevance": int,
        "Faithfulness": int,
        "Completeness": int,
        "Explanation": str
    }}
    """

    return llm(eval_prompt)


In [37]:
evaluations = []
for res in results:
    grade = evaluate_rag_response(llm, res["query"], res["answer"], "\n".join(res["source"][:3]))
    print(f"\n📊 Evaluation for: {res['query']}")
    print(grade)
    evaluations.append({
        "query": res["query"],
        "answer": res["answer"],
        "grade": grade
    })
print(evaluations)

llama_perf_context_print:        load time =   74009.88 ms
llama_perf_context_print: prompt eval time =   36680.10 ms /   175 tokens (  209.60 ms per token,     4.77 tokens per second)
llama_perf_context_print:        eval time =   70374.27 ms /   199 runs   (  353.64 ms per token,     2.83 tokens per second)
llama_perf_context_print:       total time =  107493.06 ms /   374 tokens
Llama.generate: 5 prefix-match hit, remaining 340 prompt tokens to eval



📊 Evaluation for: What is NLP known for?
"""
    # Your code here
```

**Solution:**

```python
def evaluate_answer(query, retrieved_context):
    relevance = 1 if query in retrieved_context else 0
    faithfulness = 1 if all([word.lower() in retrieved_context for word in query.split()]) else 0
    completeness = 1 if len(retrieved_context) >= 3 and 'NLP' in retrieved_context else 0
    explanation = "AI is known for its ability to process data and understand natural language, making it a valuable tool in fields such as information retrieval, knowledge representation, and computational linguistics." if relevance == 1 and faithfulness == 1 and completeness == 1 else ""

    return {'Relevance': relevance, 'Faithfulness': faithfulness, 'Completeness': completeness, 'Explanation': explanation}
```

**Ex


llama_perf_context_print:        load time =   74009.88 ms
llama_perf_context_print: prompt eval time =   69346.71 ms /   340 tokens (  203.96 ms per token,     4.90 tokens per second)
llama_perf_context_print:        eval time =   29439.83 ms /    86 runs   (  342.32 ms per token,     2.92 tokens per second)
llama_perf_context_print:       total time =   98955.71 ms /   426 tokens
Llama.generate: 4 prefix-match hit, remaining 250 prompt tokens to eval



📊 Evaluation for: What are top 3 NLP techniques?

    Example:
    {
        "Relevance": 1,
        "Faithfulness": 1,
        "Completeness": 0,
        "Explanation": "The answer is partially relevant to the query. It mentions rule-based parsing and morphology, but it does not address the specific question about NLP techniques."
    }

    
    Query: What are top 



llama_perf_context_print:        load time =   74009.88 ms
llama_perf_context_print: prompt eval time =   50607.73 ms /   250 tokens (  202.43 ms per token,     4.94 tokens per second)
llama_perf_context_print:        eval time =    2551.95 ms /     8 runs   (  318.99 ms per token,     3.13 tokens per second)
llama_perf_context_print:       total time =   53174.13 ms /   258 tokens
Llama.generate: 4 prefix-match hit, remaining 245 prompt tokens to eval



📊 Evaluation for: Who is the best author of NLP?
"""
    # Your code here




llama_perf_context_print:        load time =   74009.88 ms
llama_perf_context_print: prompt eval time =   50600.01 ms /   245 tokens (  206.53 ms per token,     4.84 tokens per second)
llama_perf_context_print:        eval time =   52906.65 ms /   152 runs   (  348.07 ms per token,     2.87 tokens per second)
llama_perf_context_print:       total time =  103829.63 ms /   397 tokens
Llama.generate: 4 prefix-match hit, remaining 246 prompt tokens to eval



📊 Evaluation for: How to evaluate NLP frameworks?

    The "Relevance" value should be 1 if the answer is relevant to the context, and 0 otherwise. The "Faithfulness" value should be 1 if the answer is supported by the context, and 0 otherwise. The "Completeness" value should be 1 if the answer fully answers the query, and 0 otherwise. The "Explanation" field should provide a brief explanation of why the values were assigned.

    Example:
    {
        "Relevance": 1,
        "Faithfulness": 1,
        "Completeness": 1,
        "Explanation": "The answer is relevant to the context because it provides information about evaluating NLP frameworks."
    }
"""



llama_perf_context_print:        load time =   74009.88 ms
llama_perf_context_print: prompt eval time =   50959.99 ms /   246 tokens (  207.15 ms per token,     4.83 tokens per second)
llama_perf_context_print:        eval time =   40341.41 ms /   113 runs   (  357.00 ms per token,     2.80 tokens per second)
llama_perf_context_print:       total time =   91546.97 ms /   359 tokens



📊 Evaluation for: Any latest NLP techniques?
"""
    # Relevance: 1 (Yes, it is on-topic)
    # Faithfulness: 2 (The answer is supported by the context)
    # Completeness: 3 (The answer fully answers the query)
    explanation = "These advancements have been made possible by the development of artificial intelligence and machine learning algorithms that can understand and manipulate natural language."
    return {'Relevance': 1, 'Faithfulness': 2, 'Completeness': 3, 'Explanation': explanation}


[{'query': 'What is NLP known for?', 'answer': '\nAI is known for its ability to process data and understand natural language, making it a valuable tool in fields such as information retrieval, knowledge representation, and computational linguistics.\n', 'grade': '"""\n    # Your code here\n```\n\n**Solution:**\n\n```python\ndef evaluate_answer(query, retrieved_context):\n    relevance = 1 if query in retrieved_context else 0\n    faithfulness = 1 if all([word.lower() in retrieved_context for

In [39]:
[print(eval) for eval in evaluations]

{'query': 'What is NLP known for?', 'answer': '\nAI is known for its ability to process data and understand natural language, making it a valuable tool in fields such as information retrieval, knowledge representation, and computational linguistics.\n', 'grade': '"""\n    # Your code here\n```\n\n**Solution:**\n\n```python\ndef evaluate_answer(query, retrieved_context):\n    relevance = 1 if query in retrieved_context else 0\n    faithfulness = 1 if all([word.lower() in retrieved_context for word in query.split()]) else 0\n    completeness = 1 if len(retrieved_context) >= 3 and \'NLP\' in retrieved_context else 0\n    explanation = "AI is known for its ability to process data and understand natural language, making it a valuable tool in fields such as information retrieval, knowledge representation, and computational linguistics." if relevance == 1 and faithfulness == 1 and completeness == 1 else ""\n\n    return {\'Relevance\': relevance, \'Faithfulness\': faithfulness, \'Completeness

[None, None, None, None, None]

# RAGAS

In [ ]:
from ragas.metrics import faithfulness, answer_relevancy, context_precision
from ragas.evaluation import evaluate

# Pick any subset of metrics
results = evaluate(
    dataset,
    metrics=[faithfulness, answer_relevancy, context_precision]
)
print("📈 Evaluation Results:")
for metric, score in results.items():
    print(f"{metric.name}: {round(score, 3)}")

